In [ ]:
import math
import numpy as np
import pandas as pd
from shapely import affinity
from shapely.geometry import Polygon, Point
from shapely.ops import unary_union
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

# Tree definition
def create_tree_polygon(x, y, angle_deg):
    """Create tree polygon at given position and angle."""
    # Tree vertices (relative to center at origin)
    trunk_w = 0.15
    trunk_h = 0.2
    base_w = 0.7
    mid_w = 0.4
    top_w = 0.25
    tip_y = 0.8
    tier_1_y = 0.5
    tier_2_y = 0.25
    base_y = 0.0
    trunk_bottom_y = -trunk_h
    
    vertices = [
        (0.0, tip_y),
        (top_w/2, tier_1_y),
        (top_w/4, tier_1_y),
        (mid_w/2, tier_2_y),
        (mid_w/4, tier_2_y),
        (base_w/2, base_y),
        (trunk_w/2, base_y),
        (trunk_w/2, trunk_bottom_y),
        (-trunk_w/2, trunk_bottom_y),
        (-trunk_w/2, base_y),
        (-base_w/2, base_y),
        (-mid_w/4, tier_2_y),
        (-mid_w/2, tier_2_y),
        (-top_w/4, tier_1_y),
        (-top_w/2, tier_1_y),
    ]
    
    # Create polygon
    poly = Polygon(vertices)
    
    # Rotate
    poly = affinity.rotate(poly, angle_deg, origin=(0, 0))
    
    # Translate
    poly = affinity.translate(poly, x, y)
    
    return poly

def validate_packing(trees):
    """
    Validate that no trees overlap.
    Returns (is_valid, overlapping_indices)
    """
    polys = [t['poly'] for t in trees]
    n = len(polys)
    
    # Check for overlaps
    # We can use STRtree for efficiency if n is large, but for n<=200 O(n^2) is fine (200^2 = 40000 checks, fast enough)
    # Actually, let's just do the simple double loop for robustness first.
    
    for i in range(n):
        for j in range(i + 1, n):
            # Use a small buffer to avoid floating point issues with touching trees?
            # The competition says "Submissions with any overlapping trees will throw an error."
            # Touching is allowed.
            if polys[i].intersects(polys[j]):
                # Intersection includes touching. We need to check if area of intersection is > 0
                # or if they just touch.
                # shapely.intersects returns True if they touch.
                # We want to allow touching.
                
                # Check intersection area
                intersection = polys[i].intersection(polys[j])
                if intersection.area > 1e-9: # Tolerance for float errors
                    return False, (i, j)
                    
    return True, None

def calculate_score_exact(trees, n):
    polys = [t['poly'] for t in trees]
    union = unary_union(polys)
    min_x, min_y, max_x, max_y = union.bounds
    side = max(max_x - min_x, max_y - min_y)
    return side**2 / n

print("Setup complete")